In [ ]:
# !pip install PytorchCML

In [1]:

import sys
sys.path.append("../../src/")

from itertools import product

from PytorchCML import losses, models, samplers, evaluators, trainers
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [28]:
def svd_init(M, dim):
    svd = TruncatedSVD(n_components=10)
    U_ = svd.fit_transform(M)
    V_ = svd.components_
    S = np.diag(svd.singular_values_)
    U = np.dot(U_, np.sqrt(S))
    V = np.dot(np.sqrt(S), V_)

    return U, V

In [2]:
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
  sep='\t', header=None, index_col=None,
)
movielens.columns = ["user_id", "item_id", "rating", "timestamp"]
movielens.user_id -= 1
movielens.item_id -= 1
movielens.rating = (movielens >= 4).astype(int)
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

train, test = train_test_split(movielens)


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values


In [58]:
X = csr_matrix((np.ones(train_set.shape[0]), (train_set[:,0], train_set[:,1])))
U, V = svd_init(X.A*2-1, 10) 

In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr = 1e-3
n_dim = 10
model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=None,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T)
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.LogitPairwiseLoss().to(device)
sampler = samplers.BaseSampler(train_set, device=device,n_neg_samples=5, batch_size=1024)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(torch.LongTensor(test_set).to(device), score_function_dict, ks=[3])
trainer = trainers.MFTrainer(model, optimizer, criterion, sampler)


In [73]:
trainer.fit(n_batch=50, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=5)

100%|██████████| 943/943 [00:14<00:00, 63.98it/s]


In [74]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,epoch,loss
0,0.323769,0.172967,0.049898,0,NaN
0,0.367278,0.183024,0.054518,5,90.533422
0,0.440782,0.202884,0.071053,10,57.645286
0,0.446070,0.225799,0.068742,15,33.645416
0,0.423904,0.229246,0.066953,20,20.807433
